In [1]:
import simulator as sm
import os
import numpy as np
from datetime import date 
import matplotlib.pyplot as plt
import astropy.units as U
from casatasks import exportfits, simobserve, tclean
times = [4.27 * 60, 1.85 * 60, 4.88 * 50, 5.15 * 60,  5.15 * 60, 3.47 * 60, 14.25 * 60]
crop = False
flatten = True
cycle = 3
antenna_config = 4
band = 3
n_pxs = 256
inbright = 10
totaltime = 2 * 60
integration = 10
if crop == True:
    output_dir = 'sim_cycle{}_antennaconfig{}_band{}_brightness{}_totaltime{}_scantime{}_cropped_to{}'.format(cycle, antenna_config, band, inbright, totaltime, integration, n_pxs)
elif n_pxs is None and crop == False:
    output_dir = 'sim_cycle{}_antennaconfig{}_band{}_brightness{}_totaltime{}_scantime{}'.format(cycle, antenna_config, band, inbright, totaltime, integration)
else:
    output_dir = 'sim_cycle{}_antennaconfig{}_band{}_brightness{}_totaltime{}_scantime{}_bounded_to{}'.format(cycle, antenna_config, band, inbright, totaltime, integration, n_pxs)

main_path = '/home/astro/Documents/GitHub/ALMASim'
data_dir = '/home/astro/Documents/casa-data'
project = 'sims'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
output_dir = os.path.join(data_dir, output_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
plot_dir = os.path.join(output_dir, "plots")
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
cycle_name = 'cycle{}'.format(int(cycle))
antenna_name = 'alma.cycle{}.0.{}'.format(cycle, antenna_config)
antennalist = os.path.join(main_path, "antenna_config", cycle_name, antenna_name + '.cfg')
spatial_resolution = sm.get_spatial_resolution(band, antenna_config)
print('Spatial Resolution:', spatial_resolution)
central_freq= sm.get_band_central_freq(band)
print('Central Frequency: ', central_freq)
max_baseline = sm.get_max_baseline_from_antenna_config(antennalist)
central_freq = sm.get_band_central_freq(band)
beam_size = sm.compute_beam_size_from_max_baseline(max_baseline, central_freq)
fov = sm.get_fov([band])[0]
cell_size = beam_size / 5

if n_pxs is None:
    n_px = sm.closest_power_of_2(int(fov / cell_size))
elif n_pxs is not None and crop is False:
    n_px = int(n_pxs)
else:
    n_px = sm.closest_power_of_2(int(fov / cell_size))
bandwidth = 1000
inwidth = 10
fwhm_x = 10 * cell_size * np.random.rand() + cell_size
fwhm_y = 10 * cell_size  * np.random.rand() + cell_size
fwhm_z = 0.1 * bandwidth * np.random.rand() + inwidth
print('FWHM X:', fwhm_x)
print('FWHM Y:', fwhm_y)
print('FWHM Z:', fwhm_z)
n_channels = int(bandwidth / inwidth)
print('Number of Channels:', n_channels)
i = 0
n_sources = 0
ra, dec = 0, 0
pa = 128
min_sep_spatial = 0
min_sep_frequency = 0
rest_frequency = 1440.4
serendipitous = False
filename = sm.generate_pointlike_skymodel(i, output_dir, rest_frequency, 
                                                   inwidth * U.MHz, fwhm_z * U.MHz,
                                                   central_freq * U.GHz, n_px, 
                                                   n_channels, ra * U.deg, dec * U.deg,
                                                   spatial_resolution * U.arcsec, plot_dir)


simobserve(
    skymodel=filename,
    project=project,
    indirection="J2000 19h30m00 -40d00m00",
    incell="{}arcsec".format(cell_size),
    incenter='{}GHz'.format(central_freq),
    inwidth="{}MHz".format(inwidth),
    inbright="{}Jy/pix".format(inbright),
    setpointings=True,
    obsmode='int',
    antennalist=antennalist,
    totaltime="{}s".format(totaltime),
    integration="{}s".format(integration),
    graphics='none',
    overwrite=True,
    thermalnoise="tsys-atm",
)
tclean(
    vis=os.path.join(project, "{}.{}.noisy.ms".format(project, antenna_name)),
    imagename=os.path.join(project, '{}.{}'.format(project, antenna_name)),
    imsize=[n_px, n_px],
    cell="{}arcsec".format(cell_size),
    specmode="cube",
    niter=0,
    fastnoise=False,
    calcpsf=True,
    pbcor=True,
    pblimit=0.2,
)
exportfits(imagename=os.path.join(project, '{}.{}.image'.format(project, antenna_name)), 
           fitsimage=os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"), overwrite=True)
exportfits(imagename=os.path.join(project, '{}.{}.skymodel'.format(project, antenna_name)), 
           fitsimage=os.path.join(output_dir, "clean_cube_" + str(i) +".fits"), overwrite=True)
clean, clean_header = sm.load_fits(os.path.join(output_dir, "clean_cube_" + str(i) +".fits"))
dirty, dirty_header = sm.load_fits(os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"))
print('Cropping and Saving Plots')
if crop == True:
    left = int((clean.shape[-1] - n_pxs) / 2)
    clean_cube = clean[:, :,  left:left+int(n_pxs), left:left+int(n_pxs)]
    dirty_cube = dirty[:, :, left:left+int(n_pxs), left:left+int(n_pxs)]
    if flatten == True:
        clean_cube = np.expand_dims(np.sum(clean_cube, axis=1), axis=1)
        dirty_cube = np.expand_dims(np.sum(dirty_cube, axis=1), axis=1)
        
    sm.write_numpy_to_fits(clean_cube, clean_header, os.path.join(output_dir, "clean_cube_" + str(i) +".fits"))
    sm.write_numpy_to_fits(dirty_cube, dirty_header, os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"))
sm.plotter(i, output_dir, plot_dir)

Spatial Resolution: 0.92
Central Frequency:  100
FWHM X: 0.42696932815473965
FWHM Y: 0.6065547057651945
FWHM Z: 82.3114618770339
Number of Channels: 100
Generating point-like source at position (127, 127, 50)
Skymodel saved to /home/astro/Documents/casa-data/sim_cycle3_antennaconfig4_band3_brightness10_totaltime120_scantime10_bounded_to256/skymodel_0.fits


2023-10-02 13:21:03	WARN	task_tclean::SIImageStore::restore (file /source/casa6/casatools/src/code/synthesis/ImagerObjects/SIImageStore.cc, line 2265)	Restoring with an empty model image. Only residuals will be processed to form the output restored image.


Cropping and Saving Plots


In [13]:
import simulator as sm
file = 'morphologies_deeplearn.hdf5'
db = sm.get_data_from_hdf(file)
db.info()
print(db['SubhaloID'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12535 entries, 0 to 12534
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   P_Late      12535 non-null  float64
 1   P_S0        12535 non-null  float64
 2   P_Sab       12535 non-null  float64
 3   Sigma_Late  12535 non-null  float64
 4   Sigma_S0    12535 non-null  float64
 5   Sigma_Sab   12535 non-null  float64
 6   SubhaloID   12535 non-null  float64
dtypes: float64(7)
memory usage: 685.6 KB
0             0.0
1             1.0
2             2.0
3             3.0
4             4.0
           ...   
12530    704187.0
12531    719337.0
12532    727485.0
12533    738596.0
12534    752194.0
Name: SubhaloID, Length: 12535, dtype: float64


In [15]:
basePath = '/home/astro/Documents/TNGData/TNG100-1/output/'
import h5py
import numpy as np
import simulator as sm
snapNum = 99
data_header = sm.loadHeader(basePath, snapNum)
for key, value in data_header.items():
    print(key, value)

BoxSize 75000.0
FlagDoubleprecision 0
Git_commit b'd203ec8b07c7e2bdda5f608aa0babea46d603699'
Git_date b'Thu Apr 7 14:14:27 2016 +0200'
HubbleParam 0.6774
Ngroups_ThisFile 1
Ngroups_Total 6291349
Nids_ThisFile 11830481
Nids_Total 5263038490
Nsubgroups_ThisFile 17185
Nsubgroups_Total 4371211
NumFiles 448
Omega0 0.3089
OmegaLambda 0.6911
Redshift 2.220446049250313e-16
Time 0.9999999999999998


In [26]:
import astropy.units as u
import astropy.cosmology.units as cu
from astropy.cosmology import Planck13
z = 0.001 * cu.redshift
d = z.to(u.Mpc, cu.redshift_distance(Planck13, kind="comoving"))
print(d)

4.422650863276812 Mpc


In [ ]:
with h5py.File(sm.offsetPath(basePath, snapNum), "r") as f:
    offsets = f["FileOffsets/" + gName][()]


In [1]:
# Simulate atmospherics
import simulator as sm
import os
project = '/home/astro/Documents/casa-data/atmospheric_errors/sim_0'
project_folder = 'sim_0'
cycle = 9
antenna_name = 'alma.cycle{}.0.10'.format(cycle)
cycle = 'cycle{}'.format(cycle)
print(project)
main_path = '/home/astro/Documents/GitHub/ALMASim'
scale = 0.3
ms_name = "{}.{}.noisy.ms".format(project_folder, antenna_name)
print(ms_name)
ms = os.path.join(project, ms_name)
print(ms)
antennalist = os.path.join(main_path, "antenna_config", cycle, antenna_name + '.cfg')


sm.simulate_atmospheric_noise(project, scale, ms, antennalist)

/home/astro/Documents/casa-data/atmospheric_errors/sim_0
sim_0.alma.cycle9.0.10.noisy.ms
/home/astro/Documents/casa-data/atmospheric_errors/sim_0/sim_0.alma.cycle9.0.10.noisy.ms
/home/astro/Documents/casa-data/atmospheric_errors/sim_0_atmosphere.gcal


In [5]:
gcaltable = '/home/astro/Documents/casa-data/atmospheric_errors/sim_0_atmosphere.gcal'
from casatools import table 
tb = table()
tb.open(gcaltable)

True

In [1]:
import simulator as sm
cycle = 9
for config_number in range(1, 10):
    antenna_name = 'alma.cycle{}.0.{}'.format(cycle, config_number)
    main_path = '/home/astro/Documents/GitHub/ALMASim'
    antenna_path = os.path.join(main_path, "antenna_config", 'cycle{}'.format(cycle), antenna_name + '.cfg')
    max_baseline = sm.get_max_baseline_from_antenna_config(antenna_path)
    print(max_baseline)

ModuleNotFoundError: No module named 'Hdecompose'